In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

# load in sc data

In [2]:
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells.h5ad')
adata_1 = adata_T[adata_T[:,'PDCD1'].X > 0, :]
adata_B = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_Fibro_cells.h5ad')
adata_2 = adata_B

In [3]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_2, sample_id_col='sample_id')

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


# prepare data matrix for IVtest

In [4]:
# because some samples are missing, we need to add them to the dataset
# to make sure nosamplefiltered
from scRNA_utils import add_missing_samples
import anndata
# Extract sample_ids
sample_ids_1 = set(adata_1_pseudo.obs['sample_id'])
sample_ids_2 = set(adata_2_pseudo.obs['sample_id'])

# Identify missing sample_ids
missing_sample_ids_1 = sample_ids_2 - sample_ids_1
missing_sample_ids_2 = sample_ids_1 - sample_ids_2

# Add missing sample_ids to each dataset
adata_1_pseudo = add_missing_samples(adata_1_pseudo, missing_sample_ids_1)
adata_2_pseudo = add_missing_samples(adata_2_pseudo, missing_sample_ids_2)
# Sort the obs DataFrame by sample_id in both AnnData objects
adata_1_pseudo = adata_1_pseudo[adata_1_pseudo.obs.sort_values('sample_id').index]
adata_2_pseudo = adata_2_pseudo[adata_2_pseudo.obs.sort_values('sample_id').index]

# Verify that both datasets have the same sample_ids and in the same order
assert (adata_1_pseudo.obs['sample_id'].values == adata_2_pseudo.obs['sample_id'].values).all()

# Now adata_T_pseudo and adata_Endo_pseudo have the same sample_ids in the same order

In [5]:
DEG_1 = paird_ttest(adata_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = paird_ttest(adata_2, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
DEG_1 = DEG_1[DEG_1['pval'] < 0.05]
DEG_2 = DEG_2[DEG_2['pval'] < 0.05]
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_Fibro' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')

In [7]:
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')
gene_df['treatment'].value_counts()

1    31
0    31
Name: treatment, dtype: int64

## group the DEGpairs together using L and R

In [8]:
results = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_PD1TvsFibro_fisherz_corr03.xlsx')

In [9]:
# results = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_CD4EXvsDC_fisherz_nosamplefiltered.xlsx')
# results = results.drop(columns=['Unnamed: 0'])
# # Remove the suffix '_X' from g1 and g2
results['g1'] = results['g1'].str.replace(r'_\w+', '', regex=True)
results['g2'] = results['g2'].str.replace(r'_\w+', '', regex=True)

In [10]:
results

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,0,IL7R,RASD1,ANXA1,EDNRB,0.327342,0.516109,0.127908
1,1,IL7R,RASD1,APP,EDNRB,0.061066,0.516109,0.241098
2,2,ZFP36L2,RASD1,ANXA1,EDNRB,0.083744,0.495170,0.223252
3,3,DDIT4,MT1M,CCL5,SDC1,0.060150,0.483293,0.241923
4,4,TNFSF8,RASD1,ANXA1,EDNRB,0.087501,0.480192,0.220676
...,...,...,...,...,...,...,...,...
3645,3645,FASLG,VMP1,CD40LG,ITGA5,0.253427,-0.421186,-0.148846
3646,3646,SLC2A3,RGS1,CXCL11,P2RY14,0.050009,-0.425993,-0.251812
3647,3647,SLC2A3,RGS1,CXCL13,CXCR4,0.111243,-0.425993,-0.206127
3648,3648,FASLG,RGS2,CALR,ITGA5,0.149199,-0.444160,-0.187160


In [11]:
# only keep the significant results with correlation > 0.3 or < -0.3
results = results[(results['g1vsg2_correlation'] > 0.3)]

In [12]:
results

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,0,IL7R,RASD1,ANXA1,EDNRB,0.327342,0.516109,0.127908
1,1,IL7R,RASD1,APP,EDNRB,0.061066,0.516109,0.241098
2,2,ZFP36L2,RASD1,ANXA1,EDNRB,0.083744,0.495170,0.223252
3,3,DDIT4,MT1M,CCL5,SDC1,0.060150,0.483293,0.241923
4,4,TNFSF8,RASD1,ANXA1,EDNRB,0.087501,0.480192,0.220676
...,...,...,...,...,...,...,...,...
2277,2277,SLA,PDK4,XCL1,LTB4R,0.050529,0.300053,0.251267
2278,2278,SLA,PDK4,XCL1,CHRM3,0.074349,0.300053,0.230115
2279,2279,SLA,PDK4,TNF,PCDH7,0.088801,0.300053,0.219805
2280,2280,SLA,PDK4,TNF,FAT1,0.069693,0.300053,0.233773


In [13]:
calculate_g1_L_correlation(results, adata_1_pseudo)
calculate_g2_R_correlation(results, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:982: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['g1vsL_correlation'] = 0
/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['g2vsR_correlation'] = 0


,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,g1vsL_correlation,g2vsR_correlation
0,0,IL7R,RASD1,ANXA1,EDNRB,0.327342,0.516109,0.127908,0.486636,0.789881
1,1,IL7R,RASD1,APP,EDNRB,0.061066,0.516109,0.241098,0.285446,0.789881
2,2,ZFP36L2,RASD1,ANXA1,EDNRB,0.083744,0.495170,0.223252,0.387030,0.789881
3,3,DDIT4,MT1M,CCL5,SDC1,0.060150,0.483293,0.241923,-0.438881,-0.614971
4,4,TNFSF8,RASD1,ANXA1,EDNRB,0.087501,0.480192,0.220676,0.164258,0.789881
...,...,...,...,...,...,...,...,...,...,...
2277,2277,SLA,PDK4,XCL1,LTB4R,0.050529,0.300053,0.251267,-0.350066,-0.342322
2278,2278,SLA,PDK4,XCL1,CHRM3,0.074349,0.300053,0.230115,-0.350066,-0.224463
2279,2279,SLA,PDK4,TNF,PCDH7,0.088801,0.300053,0.219805,-0.283085,-0.586099
2280,2280,SLA,PDK4,TNF,FAT1,0.069693,0.300053,0.233773,-0.283085,-0.342086


In [14]:
# remove the rows with g1vsL_correlation < 0.3 
results = results[(results['g1vsL_correlation'] > 0.3) & (results['g1vsL_correlation'] < 0.99)]
# remove the rows with g2vsR_correlation < 0.3 
results = results[(results['g2vsR_correlation'] > 0.3) & (results['g2vsR_correlation'] < 0.99)]

In [15]:
# get the unique (L,R) pairs in results
lr_pairs = results[['L', 'R']]
# remove the LR pairs repeated less than 3 times in the lr_pairs
lr_pairs = lr_pairs.groupby(['L', 'R']).filter(lambda x: len(x) >= 2)
lr_pairs = lr_pairs.drop_duplicates()
lr_pairs

,L,R
0,ANXA1,EDNRB
5,AREG,ICAM1
12,CXCL13,CXCR4
15,ANXA1,LPAR2
22,ITGB1,ITGA5
25,IL2,NGFR
30,NMB,EDNRB
36,ANXA1,AGTR1
69,GZMB,NOTCH1
72,CXCL13,S1PR3


In [16]:
import pandas as pd
grouped_results = results.groupby(["L", "R"])

# Display grouped data
grouped_dict = { (ligand, receptor): group for (ligand, receptor), group in grouped_results }

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
grouped_data = []
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1 and GEM_g2
    # and where the overlap is below the maximum allowable threshold
    if len(gem1) > min_gene_num and len(gem2) > min_gene_num:
        temp_data.append({
            "L": ligand,
            "R": receptor,
            "g1": gem1,
            "g2": gem2,
            "Num_genes_g1": len(gem1),
            "Num_genes_g2": len(gem2)
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)

# Function to find overlapping rows
def find_overlapping_rows(df, column_name):
    overlap_info = []
    for index, row in df.iterrows():
        current_set = set(row[column_name])
        overlapping_rows = []
        for other_index, other_row in df.iterrows():
            if index != other_index:
                other_set = set(other_row[column_name])
                overlap = len(current_set & other_set)
                if overlap > len(current_set) / 3: # Set the threshold for overlap
                    overlapping_rows.append(other_index)
        overlap_info.append(overlapping_rows)
    return overlap_info

# Add columns for overlapping rows in GEM_g1 and GEM_g2
grouped_df["Overlapping_g1"] = find_overlapping_rows(grouped_df, "g1")
grouped_df["Overlapping_g2"] = find_overlapping_rows(grouped_df, "g2")


In [17]:
# rank each GEM_g1 by order of name inside
grouped_df['g1'] = grouped_df['g1'].apply(lambda x: sorted(x))
grouped_df['g2'] = grouped_df['g2'].apply(lambda x: sorted(x))
grouped_df

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2
0,ANXA1,AGTR1,"[CXCR4, IL7R, SESN1, TXNIP]","[ADAMTS15, CREB5, CTGF, CYR61, DUSP1, EGR1, FK...",4,18,"[1, 3, 7]","[1, 7]"
1,ANXA1,EDNRB,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36L2]","[ADAMTS1, ADAMTS15, APOLD1, ARID5B, CEBPD, EGR...",6,19,"[0, 3, 7]","[0, 7]"
2,AREG,ICAM1,"[CCNH, CEMIP2, HMGB2, NFKBIA, RGS1, SRGN, TGIF...","[ATF3, CEBPB, DDIT4, ERRFI1, GLUL, NEDD9, NFKB...",10,8,[],[4]
3,CD40LG,ITGA5,"[CRYBG1, CXCR4, NFKBIA, SESN1, SOCS1, TSC22D3,...","[CTGF, CYR61, JUND, MMP19, MT1E, RGS2, SERPINE...",9,8,"[1, 7, 8]","[5, 8]"
4,CXCL13,S1PR3,"[CBLB, CTLA4, NEAT1, PELI1, SLA]","[ADAMTS1, ATF3, ERRFI1, KCNE4, MT1A, NFKBIZ, R...",5,9,[6],"[2, 6]"
5,CYR61,ITGA5,"[CXCR4, ERN1, ZFP36L2]","[CTGF, JUND, KLF6, RGS2, SERPINE1]",3,5,"[1, 3, 7]",[3]
6,GZMB,NOTCH1,"[ANKRD28, CTLA4, HMGB2, NEAT1, SLA]","[ATF3, ERRFI1, SGK1, TNFAIP3]",5,4,[4],"[2, 4]"
7,IL2,NGFR,"[CD55, CXCR4, GLUL, IL7R, NFKBIA, PARP8, SESN1...","[ADAMTS1, ADAMTS15, EGR1, GEM, MT1M, MT1X, MYC...",12,12,"[1, 3]","[0, 1]"
8,XCL1,AGTR1,"[CD55, SESN1, SOCS1, ZFP36, ZFP36L2]","[CTGF, CYR61, GADD45B, GLUL, MMP19, TSC22D3]",5,6,"[1, 3, 7, 9]","[0, 3]"
9,XCL1,EDNRB,"[CD55, SESN1, ZFP36L2]","[GLUL, NFKBIA, NR4A1, RASD1]",3,4,"[1, 3, 7, 8]","[0, 1, 7]"


In [18]:
#calculate the correlation between g1 and g2
calculate_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)
#calculate the residuals correlation between g1, g2 and LR
calculate_residuals_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,ANXA1,AGTR1,"[CXCR4, IL7R, SESN1, TXNIP]","[ADAMTS15, CREB5, CTGF, CYR61, DUSP1, EGR1, FK...",4,18,"[1, 3, 7]","[1, 7]",0.620324,0.497368
1,ANXA1,EDNRB,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36L2]","[ADAMTS1, ADAMTS15, APOLD1, ARID5B, CEBPD, EGR...",6,19,"[0, 3, 7]","[0, 7]",0.620971,0.417610
2,AREG,ICAM1,"[CCNH, CEMIP2, HMGB2, NFKBIA, RGS1, SRGN, TGIF...","[ATF3, CEBPB, DDIT4, ERRFI1, GLUL, NEDD9, NFKB...",10,8,[],[4],0.621105,0.491555
3,CD40LG,ITGA5,"[CRYBG1, CXCR4, NFKBIA, SESN1, SOCS1, TSC22D3,...","[CTGF, CYR61, JUND, MMP19, MT1E, RGS2, SERPINE...",9,8,"[1, 7, 8]","[5, 8]",0.642821,0.520436
4,CXCL13,S1PR3,"[CBLB, CTLA4, NEAT1, PELI1, SLA]","[ADAMTS1, ATF3, ERRFI1, KCNE4, MT1A, NFKBIZ, R...",5,9,[6],"[2, 6]",0.548889,0.372063
5,CYR61,ITGA5,"[CXCR4, ERN1, ZFP36L2]","[CTGF, JUND, KLF6, RGS2, SERPINE1]",3,5,"[1, 3, 7]",[3],0.487323,0.398280
6,GZMB,NOTCH1,"[ANKRD28, CTLA4, HMGB2, NEAT1, SLA]","[ATF3, ERRFI1, SGK1, TNFAIP3]",5,4,[4],"[2, 4]",0.552913,0.394901
7,IL2,NGFR,"[CD55, CXCR4, GLUL, IL7R, NFKBIA, PARP8, SESN1...","[ADAMTS1, ADAMTS15, EGR1, GEM, MT1M, MT1X, MYC...",12,12,"[1, 3]","[0, 1]",0.687053,0.560603
8,XCL1,AGTR1,"[CD55, SESN1, SOCS1, ZFP36, ZFP36L2]","[CTGF, CYR61, GADD45B, GLUL, MMP19, TSC22D3]",5,6,"[1, 3, 7, 9]","[0, 3]",0.631374,0.530599
9,XCL1,EDNRB,"[CD55, SESN1, ZFP36L2]","[GLUL, NFKBIA, NR4A1, RASD1]",3,4,"[1, 3, 7, 8]","[0, 1, 7]",0.539738,0.438469


In [19]:
grouped_df['GEM_1'] = 'GEM_' + grouped_df.index.astype(str) + '_T'
grouped_df['GEM_2'] = 'GEM_' + grouped_df.index.astype(str) + '_Fibro'

In [20]:
# Define a function to append '_T' to each gene name
def append_suffix(gene_list, suffix='_T'):
    # Ensure it's a list and not NaN or another unexpected type
    if isinstance(gene_list, list):
        return [gene + suffix for gene in gene_list]
    else:
        print("Unexpected format:", gene_list)
        return gene_list

# Apply the function to the GEM_g1 column to add '_T' to each gene name
DEG_pairs_T = grouped_df.copy()
DEG_pairs_T['g1'] = grouped_df['g1'].apply(lambda x: append_suffix(x, '_T'))

# Similarly for GEM_g2 if needed
DEG_pairs_T['g2'] = grouped_df['g2'].apply(lambda x: append_suffix(x, '_Fibro'))
# remove the genes not in gene_df from each row in DEG_pairs_T but keep the row
DEG_pairs_T['g1'] = DEG_pairs_T['g1'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])
DEG_pairs_T['g2'] = DEG_pairs_T['g2'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])


# add GEM_1_T, GEM_2_T and so on to gene_df
for i, row in DEG_pairs_T.iterrows():
    GEM_1 = DEG_pairs_T.loc[i, 'g1']
    GEM_2 = DEG_pairs_T.loc[i, 'g2']
    gene_df['GEM_' + str(i) + '_T'] = gene_df[GEM_1].mean(axis=1)
    gene_df['GEM_' + str(i) + '_Fibro'] = gene_df[GEM_2].mean(axis=1)
gene_df

,AC004687.1_T,AC004817.3_T,AC004951.1_T,AC006369.1_T,AC007038.2_T,AC015813.1_T,AC016831.5_T,AC021188.1_T,AC023157.3_T,AC025164.1_T,...,GEM_5_T,GEM_5_Fibro,GEM_6_T,GEM_6_Fibro,GEM_7_T,GEM_7_Fibro,GEM_8_T,GEM_8_Fibro,GEM_9_T,GEM_9_Fibro
sample_id,,,,,,,,,,,,,,,,,,,,,
BIOKEY_10_On,309.215912,189.972443,46.985409,88.311661,13.239152,40.373478,43.434666,111.079369,77.687202,379.605682,...,2833.618408,1053.905151,2086.694092,516.104492,2374.861084,763.672302,2490.889648,1563.485229,1855.594116,1009.782837
BIOKEY_10_Pre,563.980347,44.845234,55.078514,81.338425,5.761608,25.467459,9.838195,82.176804,36.126411,164.164597,...,1742.723022,830.064636,1229.703735,318.615997,1429.902954,405.460785,1534.217773,1144.753784,1205.033691,617.393555
BIOKEY_11_On,268.598755,148.075867,37.323822,188.380936,10.964890,28.348480,45.273457,68.983269,41.890564,551.513916,...,3522.106201,1439.154175,2053.829590,693.194763,2775.434814,1050.382690,2780.214844,1727.447876,2192.491943,1255.110474
BIOKEY_11_Pre,391.792786,26.553766,51.684788,149.571411,0.000000,17.950109,23.245640,79.012733,89.163200,415.800201,...,2470.418945,967.328979,1287.367432,384.402893,1696.640015,849.439636,2048.623535,1218.162720,1620.964478,664.350830
BIOKEY_12_On,329.073425,308.417725,18.504068,184.849304,5.138805,24.086304,120.066422,34.435535,70.810493,540.817444,...,3559.000977,1432.423096,2045.738525,863.564819,3035.210205,1249.528320,2594.039795,1729.485718,2084.757324,1526.045654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIOKEY_7_Pre,420.561005,0.000000,0.000000,1083.950928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4378.458984,1171.375366,569.193359,184.070053,2547.547607,659.185364,2891.294434,1406.522583,2812.236328,654.124573
BIOKEY_8_On,269.902893,409.661530,45.042774,241.828293,0.000000,0.000000,82.949020,111.544533,60.184547,431.229431,...,2874.504639,976.683472,1675.131226,1945.003662,2454.752686,965.496765,2710.167236,1362.179932,1990.793945,1214.861328
BIOKEY_8_Pre,850.882141,0.000000,0.000000,193.594498,0.000000,0.000000,236.877213,0.000000,0.000000,245.685928,...,2464.437500,675.758606,1353.617798,258.479034,2110.072510,1313.366577,2397.138428,1376.426758,1842.746094,1640.543457


In [21]:
# add GEM_1_T, GEM_2_T and so on to adata
DEG_pairs = grouped_df
for i, row in DEG_pairs.iterrows():
    GEM_1 = DEG_pairs.loc[i, 'g1']
    GEM_2 = DEG_pairs.loc[i, 'g2']
    # add GEMs to adata_1_pseudo and adata_2_pseudo
    adata_1_pseudo.obs['GEM_' + str(i) + '_T'] = adata_1_pseudo[:, GEM_1].X.mean(axis=1)
    adata_2_pseudo.obs['GEM_' + str(i) + '_Fibro'] = adata_2_pseudo[:, GEM_2].X.mean(axis=1)
    # add GEMs to adata_1 and adata_2
    adata_1.obs['GEM_' + str(i) + '_T'] = adata_1[:, GEM_1].X.mean(axis=1)
    adata_2.obs['GEM_' + str(i) + '_Fibro'] = adata_2[:, GEM_2].X.mean(axis=1)

/tmp/ipykernel_1602517/3186404608.py:7: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1_pseudo.obs['GEM_' + str(i) + '_T'] = adata_1_pseudo[:, GEM_1].X.mean(axis=1)
/tmp/ipykernel_1602517/3186404608.py:8: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_2_pseudo.obs['GEM_' + str(i) + '_Fibro'] = adata_2_pseudo[:, GEM_2].X.mean(axis=1)
/tmp/ipykernel_1602517/3186404608.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1.obs['GEM_' + str(i) + '_T'] = adata_1[:, GEM_1].X.mean(axis=1)


In [27]:
CIT_GEM_test(grouped_df, adata_1_pseudo, adata_2_pseudo, gene_df, method="fisherz")

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,GEM_1,GEM_2,pValue
0,ANXA1,AGTR1,"[CXCR4, IL7R, SESN1, TXNIP]","[ADAMTS15, CREB5, CTGF, CYR61, DUSP1, EGR1, FK...",4,18,"[1, 3, 7]","[1, 7]",0.620324,0.497368,GEM_0_T,GEM_0_Fibro,0.000032
1,ANXA1,EDNRB,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36L2]","[ADAMTS1, ADAMTS15, APOLD1, ARID5B, CEBPD, EGR...",6,19,"[0, 3, 7]","[0, 7]",0.620971,0.417610,GEM_1_T,GEM_1_Fibro,0.000705
2,AREG,ICAM1,"[CCNH, CEMIP2, HMGB2, NFKBIA, RGS1, SRGN, TGIF...","[ATF3, CEBPB, DDIT4, ERRFI1, GLUL, NEDD9, NFKB...",10,8,[],[4],0.621105,0.491555,GEM_2_T,GEM_2_Fibro,0.000042
3,CD40LG,ITGA5,"[CRYBG1, CXCR4, NFKBIA, SESN1, SOCS1, TSC22D3,...","[CTGF, CYR61, JUND, MMP19, MT1E, RGS2, SERPINE...",9,8,"[1, 7, 8]","[5, 8]",0.642821,0.520436,GEM_3_T,GEM_3_Fibro,0.000011
4,CXCL13,S1PR3,"[CBLB, CTLA4, NEAT1, PELI1, SLA]","[ADAMTS1, ATF3, ERRFI1, KCNE4, MT1A, NFKBIZ, R...",5,9,[6],"[2, 6]",0.548889,0.372063,GEM_4_T,GEM_4_Fibro,0.002917
5,CYR61,ITGA5,"[CXCR4, ERN1, ZFP36L2]","[CTGF, JUND, KLF6, RGS2, SERPINE1]",3,5,"[1, 3, 7]",[3],0.487323,0.398280,GEM_5_T,GEM_5_Fibro,0.001324
6,GZMB,NOTCH1,"[ANKRD28, CTLA4, HMGB2, NEAT1, SLA]","[ATF3, ERRFI1, SGK1, TNFAIP3]",5,4,[4],"[2, 4]",0.552913,0.394901,GEM_6_T,GEM_6_Fibro,0.001471
7,IL2,NGFR,"[CD55, CXCR4, GLUL, IL7R, NFKBIA, PARP8, SESN1...","[ADAMTS1, ADAMTS15, EGR1, GEM, MT1M, MT1X, MYC...",12,12,"[1, 3]","[0, 1]",0.687053,0.560603,GEM_7_T,GEM_7_Fibro,0.000001
8,XCL1,AGTR1,"[CD55, SESN1, SOCS1, ZFP36, ZFP36L2]","[CTGF, CYR61, GADD45B, GLUL, MMP19, TSC22D3]",5,6,"[1, 3, 7, 9]","[0, 3]",0.631374,0.530599,GEM_8_T,GEM_8_Fibro,0.000007
9,XCL1,EDNRB,"[CD55, SESN1, ZFP36L2]","[GLUL, NFKBIA, NR4A1, RASD1]",3,4,"[1, 3, 7, 8]","[0, 1, 7]",0.539738,0.438469,GEM_9_T,GEM_9_Fibro,0.000341


In [24]:
# save the grouped_df to excel
grouped_df.to_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_PD1vsFibro_grouped_0920.xlsx')